In [88]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import skimage
from importlib import reload
import folded_dataset
reload(folded_dataset)
from sklearn.neighbors import KNeighborsClassifier

print('Load features...')

features_path = '/scr/zchen/datasets/morphem_70k_2.0/Allen/pretrained_resnet18_features.npy'
df_path = '/scr/zchen/datasets/morphem_70k_2.0/Allen/enriched_meta.csv'

features = np.load(features_path)
df = pd.read_csv(df_path)

tasks = ['Task_one','Task_two']
task_labels = {'Task_one' : ['Mitotic'],
               'Task_two' : ['Mitotic']}
train_indices = np.where(df['train_test_split'] == 'train')[0]
all_test_indices = [np.where((df['train_test_split'] == 'test') & (df[task]))[0] for task in tasks]

train_X = features[train_indices]
test_Xs = [features[test_indices] for test_indices in all_test_indices]

task_Ys = [df[task_label].values for task_label in [task_labels[key] for key in tasks]]
train_Ys = [task_Ys[task_ind][train_indices] for task_ind in range(len(tasks))]
test_Ys = [task_Ys[task_ind][test_indices] for task_ind, test_indices in enumerate(all_test_indices)]

import numpy as np
import faiss

class FaissKNeighbors:
    def __init__(self, k=5):
        self.index = None
        self.y = None
        self.k = k

    def fit(self, X, y):
        self.index = faiss.IndexFlatL2(X.shape[1])
        self.index.add(X.astype(np.float32))
        self.y = y

    def predict(self, X):
        distances, indices = self.index.search(X.astype(np.float32), k=self.k)
        votes = self.y[indices]
        predictions = np.array([np.argmax(np.bincount(x)) for x in votes])
        return predictions

print('Train classifiers...')
accuracies = []
for task_ind, task in enumerate(tasks):
    knn = FaissKNeighbors(k=5)
    knn.fit(train_X, train_Ys[task_ind][:,0])
    predictions = knn.predict(test_Xs[task_ind])
    ground_truth = test_Ys[task_ind][:,0]
    accuracy = np.mean(predictions == ground_truth)
    accuracies.append(accuracy)

print('Results:')
for task_ind, task in enumerate(tasks):
    print(f'Accuracy in task {task} was {accuracies[task_ind]}')    